In [152]:
import numpy as np # for numerical operations 
import pandas as pd # for handling input data 
import matplotlib.pyplot as plt # for data visualization 
import seaborn as sns # for data visualization 
from sklearn.model_selection import train_test_split 

In [153]:
from sklearn.datasets import load_iris

In [160]:
dat=load_iris()

In [165]:
df=pd.DataFrame(data=dat.data,columns=dat.feature_names)
df_targets=dat.target

In [166]:
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [167]:
X=df
y=df_targets

In [188]:
X_train, X_test ,y_train, y_test  = train_test_split(X,y, random_state=42,test_size=0.20,stratify=y, shuffle=True)

In [189]:
classes=np.unique(y)

In [190]:
X_dict={}
for c in classes:
    X_dict[c]=X_train[y_train==c]

In [191]:
mean_dict={}
std_dict={}
for k,v in X_dict.items():
    mean_dict[k] =np.mean(v,axis=0)
    std_dict[k] =np.std(v,axis=0)

In [192]:
def gaussian_likelihood(x,mean,std):
    eps=1e-6 # to avoid dividing by zero
    P=(1/((std+eps)*np.sqrt(2*np.pi)))*np.exp(-0.5*np.square((x-mean)/(std+eps)))
    return P

In [193]:
likelihood_dict={}
for (k_mean,val_mean),(k_std,val_std) in zip (mean_dict.items(),std_dict.items()):
    likelihood_dict[k_mean] = gaussian_likelihood(X_test, val_mean, val_std)

In [194]:
# priority for each class 
prior_dict={}
for k,v in X_dict.items():
    prior_dict[k] = len(v) / len(X_train)

In [195]:
posterior_dict={}
for (k_lik,val_lik),(k_prior,val_prior) in zip (likelihood_dict.items(),prior_dict.items()):
    posterior_dict[k_lik] = np.prod(val_lik,axis=1) * val_prior

In [196]:
# convert dictionary to array of shape nclasses X samples
res=pd.DataFrame(np.array([posterior_dict[k] for k in posterior_dict]))

In [197]:
res

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.683157e-01,5.398268e-144,1.870831e-39,9.784669e-40,2.525403e-01,8.359291e-124,1.523723e-02,5.125325e-01,6.604023e-176,2.644603e-105,...,1.042173e+00,2.380221e-230,9.325784e-109,4.174832e-172,7.479885e-215,6.135798e-149,6.829260e-102,5.321493e-02,7.002318e-185,2.392300e+00
1,3.542853e-21,5.561621e-03,7.170759e-04,7.258282e-04,1.669041e-21,1.973565e-02,2.870489e-24,1.676359e-17,2.723962e-05,1.462396e-01,...,7.952119e-20,3.224978e-11,1.698788e-01,4.802072e-03,4.461178e-08,3.819331e-03,1.624792e-01,1.670242e-17,1.465001e-04,1.376308e-19
2,2.088488e-26,5.781974e-02,3.658621e-10,3.139421e-10,1.715803e-26,1.070744e-02,1.794692e-28,6.617549e-23,1.844719e-01,9.915572e-04,...,1.393765e-24,8.805413e-02,1.069746e-03,5.373684e-03,1.646003e-01,6.128621e-02,1.498717e-03,2.211909e-22,1.533922e-01,8.899662e-25


In [198]:
res_final=np.argmax(res,axis=0)

In [199]:
res_final

array([0, 2, 1, 1, 0, 1, 0, 0, 2, 1, 2, 2, 2, 1, 0, 0, 0, 1, 1, 2, 0, 2,
       1, 2, 2, 2, 1, 0, 2, 0])

In [200]:
from sklearn.metrics import accuracy_score

In [201]:
print('Test Accuracy',accuracy_score(res_final, y_test))

Test Accuracy 0.9666666666666667


# Using SKlearn

In [203]:
from sklearn.naive_bayes import GaussianNB

In [204]:
cl=GaussianNB()
cl.fit(X_train,y_train)

y_pred=cl.predict(X_test)

print('Test Accuracy',accuracy_score(y_pred, y_test))

Test Accuracy 0.9666666666666667


# Categorical Naive Bayes

In [208]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
df = pd.read_csv(url, header=None,
                 names=['buying','maint','doors','persons','lug_boot','safety','class'])

In [209]:
X=df.drop(['class'],axis=1)
y=df['class']

In [210]:
X_train, X_test ,y_train, y_test  = train_test_split(X,y, random_state=42,test_size=0.20,stratify=y, shuffle=True)

In [211]:
class_probs_dict=y_train.value_counts(normalize=True).to_dict()

In [212]:
likelihood_dict={}

In [213]:
import warnings
warnings.filterwarnings('ignore')

In [214]:
# training
for col in X_train.columns:
    likelihood_dict[col]={}
    for c in y.unique():
        subset = X_train[y==c]
        props=subset[col].value_counts(normalize=True)
        likelihood_dict[col][c]=props.to_dict()

In [215]:
# prediction
results = []

for _,row in X_test.iterrows():
    class_scores={}
    for c in class_probs_dict:
        score=np.log(class_probs_dict[c]) # class prop
        for col in row.index:
            value=row[col]
            prop=likelihood_dict.get(col).get(c).get(value,1e-6)
            score +=np.log(prop)
        class_scores[c]=score
    best_class = max(class_scores,key=class_scores.get)
    results.append(best_class)

In [216]:
print('Test Accuracy',accuracy_score(results, y_test))

Test Accuracy 0.8641618497109826


# Using SKlearn

In [240]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import OrdinalEncoder

In [241]:
X=df.drop(['class'],axis=1)
y=df['class']

In [242]:
encoder = OrdinalEncoder()
X_encoded = encoder.fit_transform(X)


In [243]:
X_train, X_test ,y_train, y_test  = train_test_split(X_encoded,y, random_state=42,test_size=0.20,stratify=y, shuffle=True)

In [244]:
cl=CategoricalNB()
cl.fit(X_train,y_train)

CategoricalNB()

In [245]:
y_pred=cl.predict(X_test)

print('Test Accuracy',accuracy_score(y_pred, y_test))

Test Accuracy 0.8641618497109826
